In [5]:
import os
import shutil
import csv
import re

In [6]:
def renamer(ref_csv, input_path, output_path): 
    
    # read reference csv
    csv_file = open(ref_csv)
    csvreader = csv.reader(csv_file)
    next(csvreader)
    names_ref = []
    for row in csvreader: 
        names_ref.append(row)
    csv_file.close()
    
    # make output dir if doesn't already exist
    if not os.path.isdir(output_path): 
        os.mkdir(output_path)
        
    # make a copy of original directory and put it in output path
    copy_data_path = output_path + "/data_copy"
    if not os.path.exists(copy_data_path): 
        os.mkdir(copy_data_path)
    shutil.copytree(input_path, copy_data_path, dirs_exist_ok=True)
    
    # get all .ab1 files and their indices
    input_dir_raw = os.listdir(input_path)
    ab_files = [x for x in input_dir_raw if x[-4:] == '.ab1']
    ab_file_indices = [int(re.findall('-\d+', x)[0][1:]) for x in ab_files]
    
    # get corresponding name for each file based on index
    def findName(ab_file_index): 
        for i in names_ref: 
            if int(i[0]) == ab_file_index: 
                return i[1]
        raise Exception("cannot find index in csv")
    
    # copy and rename each file with reference string
    for i in range(len(ab_file_indices)):
        new_path = shutil.copy2(input_path+"/"+ab_files[i], output_path)
        
        os.rename(src=new_path, dst=f"{output_path}/{findName(ab_file_indices[i])}")

In [7]:
renamer("Test Data/name_ref.csv", "Test Data/files_to_rename", 'Test Data/output_dir')